In [1]:
from features.fear_index import get_fng_features

# Daily (no leakage, 3-day lookback)
fng_day = get_fng_features("2023-09-28", "2025-09-28", interval="1d", lookback_days=3)

# 4-hour grid for the last 90 days, forward-filled from past-only smoothed daily values
fng_4h = get_fng_features("2025-06-30", "2025-09-28", interval="4h", lookback_days=3)

# 1-hour grid with a longer smoothing window (7 days)
fng_1h = get_fng_features("2025-08-01", "2025-09-28", interval="1h", lookback_days=7)



In [7]:
from features.on_chain_data import get_btc_onchain_smoothed

# 1) Daily smoothed (3-day lookback), on-chain only
df_day = get_btc_onchain_smoothed("2024-07-01", "2025-09-28", interval="1d", lookback_days=3)

# 2) 4-hour grid, smoothed on-chain + BTCUSDT price
df_4h  = get_btc_onchain_smoothed("2025-06-30", "2025-09-28", interval="4h",
                                  lookback_days=3, include_price=False)

# 3) 1-hour grid, only a couple of metrics, longer smoothing
df_1h  = get_btc_onchain_smoothed("2025-08-01", "2025-09-28", interval="1h",
                                  metrics=["n-transactions","hash-rate"],
                                  lookback_days=7, include_price=False)


/home/azureuser/maper_trader/features/on_chain_data.py:92: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return pd.date_range(start=start_ts.floor("4h"), end=end_ts, freq="4H", tz="UTC")
/home/azureuser/maper_trader/features/on_chain_data.py:94: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return pd.date_range(start=start_ts.floor("1h"), end=end_ts, freq="1H", tz="UTC")


In [8]:
df_4h.head(10)

,unique_addresses_used_smooth,confirmed_tx_per_day_smooth,output_value_per_day_btc_smooth,avg_tx_per_block_smooth,median_confirmation_time_min_smooth,avg_confirmation_time_min_smooth,total_hash_rate_ths_smooth,network_difficulty_smooth,miners_revenue_usd_smooth,total_tx_fees_btc_smooth,blockchain_size_smooth,avg_block_size_smooth
ts_utc,,,,,,,,,,,,
2025-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 20:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-01 00:00:00+00:00,521131.0,339268.0,629712.813392,2339.77931,6.85,16.949163,8.430357e+08,1.169585e+14,4.933359e+07,4.228973,668824.13727,1.377226
2025-07-01 04:00:00+00:00,521131.0,339268.0,629712.813392,2339.77931,6.85,16.949163,8.430357e+08,1.169585e+14,4.933359e+07,4.228973,668824.13727,1.377226
2025-07-01 08:00:00+00:00,521131.0,339268.0,629712.813392,2339.77931,6.85,16.949163,8.430357e+08,1.169585e+14,4.933359e+07,4.228973,668824.13727,1.377226
